# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 21 2022 1:11PM,240093,12,HH-35273,Hush Hush,Released
1,Jun 21 2022 1:11PM,240093,12,HH-35274,Hush Hush,Released
2,Jun 21 2022 1:11PM,240093,12,HH-35275,Hush Hush,Released
3,Jun 21 2022 1:11PM,240093,12,HH-35276,Hush Hush,Released
4,Jun 21 2022 1:10PM,240094,19,60016530,"GCH Granules USA, Inc.",Released
5,Jun 21 2022 1:02PM,240092,19,60015078,"GUSA Granules USA, Inc.",Released
6,Jun 21 2022 12:56PM,240090,10,Enova-10178,Emerginnova,Released
7,Jun 21 2022 12:56PM,240090,10,Enova-10179,Emerginnova,Released
8,Jun 21 2022 12:56PM,240090,10,Enova-10180,Emerginnova,Released
9,Jun 21 2022 12:38PM,240089,102,PBR-061722-01,"Senseonics, Incorporated",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
47,240089,Released,1
48,240090,Released,3
49,240092,Released,1
50,240093,Released,4
51,240094,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240089,NaN,NaN,1.0
240090,NaN,NaN,3.0
240092,NaN,NaN,1.0
240093,NaN,NaN,4.0
240094,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239992,0.0,0.0,1.0
240002,1.0,13.0,0.0
240003,0.0,0.0,1.0
240004,0.0,1.0,1.0
240006,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239992,0,0,1
240002,1,13,0
240003,0,0,1
240004,0,1,1
240006,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239992,0,0,1
1,240002,1,13,0
2,240003,0,0,1
3,240004,0,1,1
4,240006,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239992,,,1
1,240002,1,13,
2,240003,,,1
3,240004,,1,1
4,240006,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 21 2022 1:11PM,240093,12,Hush Hush
4,Jun 21 2022 1:10PM,240094,19,"GCH Granules USA, Inc."
5,Jun 21 2022 1:02PM,240092,19,"GUSA Granules USA, Inc."
6,Jun 21 2022 12:56PM,240090,10,Emerginnova
9,Jun 21 2022 12:38PM,240089,102,"Senseonics, Incorporated"
10,Jun 21 2022 12:33PM,240088,10,"Nextsource Biotechnology, LLC"
11,Jun 21 2022 12:20PM,240086,20,"ACI Healthcare USA, Inc."
28,Jun 21 2022 12:19PM,240085,20,"ACI Healthcare USA, Inc."
40,Jun 21 2022 11:58AM,240084,10,Eywa Pharma Inc.
69,Jun 21 2022 11:44AM,240081,20,"ACI Healthcare USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 21 2022 1:11PM,240093,12,Hush Hush,,,4
1,Jun 21 2022 1:10PM,240094,19,"GCH Granules USA, Inc.",,,1
2,Jun 21 2022 1:02PM,240092,19,"GUSA Granules USA, Inc.",,,1
3,Jun 21 2022 12:56PM,240090,10,Emerginnova,,,3
4,Jun 21 2022 12:38PM,240089,102,"Senseonics, Incorporated",,,1
5,Jun 21 2022 12:33PM,240088,10,"Nextsource Biotechnology, LLC",,,1
6,Jun 21 2022 12:20PM,240086,20,"ACI Healthcare USA, Inc.",,,17
7,Jun 21 2022 12:19PM,240085,20,"ACI Healthcare USA, Inc.",,,12
8,Jun 21 2022 11:58AM,240084,10,Eywa Pharma Inc.,,,29
9,Jun 21 2022 11:44AM,240081,20,"ACI Healthcare USA, Inc.",,,37


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 21 2022 1:11PM,240093,12,Hush Hush,4,,
1,Jun 21 2022 1:10PM,240094,19,"GCH Granules USA, Inc.",1,,
2,Jun 21 2022 1:02PM,240092,19,"GUSA Granules USA, Inc.",1,,
3,Jun 21 2022 12:56PM,240090,10,Emerginnova,3,,
4,Jun 21 2022 12:38PM,240089,102,"Senseonics, Incorporated",1,,
5,Jun 21 2022 12:33PM,240088,10,"Nextsource Biotechnology, LLC",1,,
6,Jun 21 2022 12:20PM,240086,20,"ACI Healthcare USA, Inc.",17,,
7,Jun 21 2022 12:19PM,240085,20,"ACI Healthcare USA, Inc.",12,,
8,Jun 21 2022 11:58AM,240084,10,Eywa Pharma Inc.,29,,
9,Jun 21 2022 11:44AM,240081,20,"ACI Healthcare USA, Inc.",37,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 21 2022 1:11PM,240093,12,Hush Hush,4,,
1,Jun 21 2022 1:10PM,240094,19,"GCH Granules USA, Inc.",1,,
2,Jun 21 2022 1:02PM,240092,19,"GUSA Granules USA, Inc.",1,,
3,Jun 21 2022 12:56PM,240090,10,Emerginnova,3,,
4,Jun 21 2022 12:38PM,240089,102,"Senseonics, Incorporated",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 21 2022 1:11PM,240093,12,Hush Hush,4.0,NaN,NaN
1,Jun 21 2022 1:10PM,240094,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
2,Jun 21 2022 1:02PM,240092,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
3,Jun 21 2022 12:56PM,240090,10,Emerginnova,3.0,NaN,NaN
4,Jun 21 2022 12:38PM,240089,102,"Senseonics, Incorporated",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 21 2022 1:11PM,240093,12,Hush Hush,4.0,0.0,0.0
1,Jun 21 2022 1:10PM,240094,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
2,Jun 21 2022 1:02PM,240092,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
3,Jun 21 2022 12:56PM,240090,10,Emerginnova,3.0,0.0,0.0
4,Jun 21 2022 12:38PM,240089,102,"Senseonics, Incorporated",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4561047,186.0,19.0,0.0
102,240089,1.0,0.0,0.0
12,240093,4.0,0.0,0.0
15,1440359,46.0,50.0,0.0
16,240070,2.0,0.0,0.0
19,1440272,5.0,15.0,3.0
20,1200336,75.0,31.0,0.0
21,239992,1.0,0.0,0.0
22,240003,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4561047,186.0,19.0,0.0
1,102,240089,1.0,0.0,0.0
2,12,240093,4.0,0.0,0.0
3,15,1440359,46.0,50.0,0.0
4,16,240070,2.0,0.0,0.0
5,19,1440272,5.0,15.0,3.0
6,20,1200336,75.0,31.0,0.0
7,21,239992,1.0,0.0,0.0
8,22,240003,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,186.0,19.0,0.0
1,102,1.0,0.0,0.0
2,12,4.0,0.0,0.0
3,15,46.0,50.0,0.0
4,16,2.0,0.0,0.0
5,19,5.0,15.0,3.0
6,20,75.0,31.0,0.0
7,21,1.0,0.0,0.0
8,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,186.0
1,102,Released,1.0
2,12,Released,4.0
3,15,Released,46.0
4,16,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,19,20,21,22
Status,,,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
Executing,19.0,0.0,0.0,50.0,0.0,15.0,31.0,0.0,0.0
Released,186.0,1.0,4.0,46.0,2.0,5.0,75.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
1,Executing,19.0,0.0,0.0,50.0,0.0,15.0,31.0,0.0,0.0
2,Released,186.0,1.0,4.0,46.0,2.0,5.0,75.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
1,Executing,19.0,0.0,0.0,50.0,0.0,15.0,31.0,0.0,0.0
2,Released,186.0,1.0,4.0,46.0,2.0,5.0,75.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()